In [1]:
import nltk


In [2]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [3]:
import json

In [4]:
import pickle

In [5]:
import numpy as np

In [6]:
from keras.models import Sequential

Using TensorFlow backend.
C:\Users\Anjali Choudhary\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Anjali Choudhary\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Anjali Choudhary\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Anjali Choudhary\anaconda3\lib\site-pa

In [7]:
from keras.layers import Dense, Activation, Dropout


In [8]:
from keras.optimizers import SGD

In [9]:
import random

In [12]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']
data_file=open('intents.json').read()
intents=json.loads(data_file)

In [13]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
       
        documents.append((w, intent['tag']))

      
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [14]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [15]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
47/47 [==============================] - 0s 7ms/step - loss: 2.2260 - accuracy: 0.1915
Epoch 2/200
47/47 [==============================] - 0s 679us/step - loss: 2.1525 - accuracy: 0.2766
Epoch 3/200
47/47 [==============================] - 0s 806us/step - loss: 2.1012 - accuracy: 0.1702
Epoch 4/200
47/47 [==============================] - 0s 728us/step - loss: 1.9997 - accuracy: 0.2979
Epoch 5/200
47/47 [==============================] - 0s 743us/step - loss: 1.9511 - accuracy: 0.3404
Epoch 6/200
47/47 [==============================] - 0s 770us/step - loss: 1.8699 - accuracy: 0.3191
Epoch 7/200
47/47 [==============================] - 0s 874us/step - loss: 1.6285 - accuracy: 0.6596
Epoch 8/200
47/47 [==============================] - 0s 832us/step - loss: 1.4965 - accuracy: 0.6383
Epoch 9/200
47/47 [==============================] - 0s 571us/step - loss: 1.5004 - accuracy: 0.5319
Epoch 10/200
47/47 [==============================] - 0s 594us/step - loss: 1.3405 - accurac

47/47 [==============================] - 0s 413us/step - loss: 0.0404 - accuracy: 1.0000
Epoch 80/200
47/47 [==============================] - 0s 470us/step - loss: 0.0541 - accuracy: 0.9787
Epoch 81/200
47/47 [==============================] - 0s 508us/step - loss: 0.0425 - accuracy: 1.0000
Epoch 82/200
47/47 [==============================] - 0s 606us/step - loss: 0.0327 - accuracy: 1.0000
Epoch 83/200
47/47 [==============================] - 0s 552us/step - loss: 0.0770 - accuracy: 0.9787
Epoch 84/200
47/47 [==============================] - 0s 528us/step - loss: 0.0331 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 575us/step - loss: 0.0580 - accuracy: 0.9787
Epoch 86/200
47/47 [==============================] - 0s 743us/step - loss: 0.0227 - accuracy: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 814us/step - loss: 0.0497 - accuracy: 0.9787
Epoch 88/200
47/47 [==============================] - 0s 747us/step - loss: 0.0195 - accuracy: 

47/47 [==============================] - 0s 573us/step - loss: 0.0205 - accuracy: 1.0000
Epoch 159/200
47/47 [==============================] - 0s 587us/step - loss: 0.0207 - accuracy: 1.0000
Epoch 160/200
47/47 [==============================] - 0s 730us/step - loss: 0.0430 - accuracy: 0.9787
Epoch 161/200
47/47 [==============================] - 0s 589us/step - loss: 0.0619 - accuracy: 0.9787
Epoch 162/200
47/47 [==============================] - 0s 552us/step - loss: 0.0318 - accuracy: 0.9787
Epoch 163/200
47/47 [==============================] - 0s 574us/step - loss: 0.0786 - accuracy: 0.9787
Epoch 164/200
47/47 [==============================] - 0s 658us/step - loss: 0.0268 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 636us/step - loss: 0.0160 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 611us/step - loss: 0.0241 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 636us/step - loss: 0.0568 - a